In [1]:
import pandas as pd
import numpy as np
import edhec_risk_kit as erk

In [2]:
hfi = erk.get_hfi_returns()

c:\ProgramData\Anaconda3\lib\site-packages\pandas\io\parsers\base_parser.py:1070: UserWarning: Parsing '31/01/1997' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return tools.to_datetime(
c:\ProgramData\Anaconda3\lib\site-packages\pandas\io\parsers\base_parser.py:1070: UserWarning: Parsing '28/02/1997' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return tools.to_datetime(
c:\ProgramData\Anaconda3\lib\site-packages\pandas\io\parsers\base_parser.py:1070: UserWarning: Parsing '31/03/1997' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return tools.to_datetime(
c:\ProgramData\Anaconda3\lib\site-packages\pandas\io\parsers\base_parser.py:1070: UserWarning: Parsing '30/04/1997' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return tools.to_datetime(
c:\ProgramData\Anaconda3\lib

In [6]:
hfi_2000 = hfi['2000':]
hfi_2000.head()

,Convertible Arbitrage,CTA Global,Distressed Securities,Emerging Markets,Equity Market Neutral,Event Driven,Fixed Income Arbitrage,Global Macro,Long/Short Equity,Merger Arbitrage,Relative Value,Short Selling,Funds Of Funds
date,,,,,,,,,,,,,
2000-01,0.0227,0.0128,0.0088,0.0077,0.0075,0.0088,0.0041,0.0021,0.0075,0.0143,0.0173,0.0427,0.0169
2000-02,0.0267,-0.0022,0.0421,0.0528,0.0253,0.0346,0.0097,0.0408,0.0699,0.0239,0.0185,-0.1340,0.0666
2000-03,0.0243,-0.0138,0.0103,0.0318,0.0134,0.0069,-0.0061,-0.0104,0.0006,0.0131,0.0163,-0.0230,0.0039
2000-04,0.0223,-0.0241,-0.0101,-0.0541,0.0168,-0.0059,-0.0006,-0.0304,-0.0201,0.0188,0.0092,0.1028,-0.0269
2000-05,0.0149,0.0114,-0.0132,-0.0433,0.0062,-0.0034,0.0107,-0.0070,-0.0097,0.0146,0.0080,0.0704,-0.0122


Monthly Parametric Gaussian VaR at the 1% level (as a +ve number) of the Distressed Securities strategy

In [8]:
round(erk.var_gaussian(hfi_2000['Distressed Securities'], 1)*100, 2)

3.14

1% VaR for the same strategy after applying the Cornish-Fisher Adjustment

In [9]:
round(erk.var_gaussian(hfi_2000['Distressed Securities'], 1, modified = True)*100, 2)

4.97

Monthly Historic VaR at the 1% level (as a +ve number) of the Distressed Securities strategy

In [10]:
round(erk.var_historic(hfi_2000['Distressed Securities'], 1)*100, 2)

4.26

For purposes of the remaining questions, use data during the 5 year period 2013-2017 (both inclusive) to estimate the expected returns as well as the covariance matrix. To be able to respond to the questions, you will need to build the MSR, EW and GMV portfolios consisting of the “Books”, “Steel”, "Oil", and "Mines" industries. Assume the risk free rate over the 5 year period is 10%.

What is the weight of Steel in the EW Portfolio?

In [15]:
ind = erk.get_ind_returns()
ind_2013_2017 = ind['2013':'2017']

er = erk.annualize_rets(ind['2013':'2017'], 12)
cov = ind['2013':'2017'].cov()

In [26]:
l = ["Books", "Oil", "Mines", "Steel"]
rf = 0.1

n = er[l].shape[0]
w_ew = np.repeat(1 / n, n)
r_ew = erk.portfolio_return(w_ew, er[l])
vol_ew = erk.portfolio_vol(w_ew, cov.loc[l, l])

In [22]:
w_ew

array([0.25, 0.25, 0.25, 0.25])

What is the weight of the largest component of the MSR portfolio? 

In [38]:
w_msr = erk.msr(rf, er[l], cov.loc[l, l])
w_msr

array([3.67414432e-14, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00])

What is the weight of the largest component of the GMV portfolio?

In [42]:
w_gmv = erk.gmv(cov.loc[l, l])
w_gmv

array([4.77024864e-01, 4.34112029e-01, 8.88631070e-02, 1.08420217e-18])

Assume two different investors invested in the GMV and MSR portfolios at the start of 2018 using the weights we just computed. Compute the annualized volatility of these two portfolios over the next 12 months of 2018? (Hint: Use the portfolio_vol code we developed in the lab and use ind[“2018”][l].cov() to compute the covariance matrix for 2018, assuming that the variable ind holds the industry returns and the variable l holds the list of industry portfolios you are willing to hold.

In [46]:
cov_2018 = ind.loc['2018'][l].cov()

port_vol_2018 = erk.portfolio_vol(w_msr, cov_2018)
port_vol_2018

0.06346230747220638

In [58]:
print(erk.annualize_vol(ind.loc['2018']['Steel'], 12))
print((port_vol_2018)*(12**0.5))

0.21983988181484185
0.2198398818148389


In [43]:
port_vol_2018_gmv = erk.portfolio_vol(w_gmv, cov_2018)
port_vol_2018_gmv

0.05477245371280368

In [59]:
(port_vol_2018_gmv)*np.sqrt(12)

0.18973734537158113